In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time

In [2]:
class handDetector():
    def __init__ (self,mode=False,maxHands=2,detectionCon=0.5,trackCon=0.5):
        self.mode=mode 
        self.maxHands=maxHands
        self.detectionCon=detectionCon
        self.trackCon=trackCon
        self.mpHands=mp.solutions.hands
        self.hands=self.mpHands.Hands(self.mode,self.maxHands,self.detectionCon,self.trackCon) 
        self.mpDraw=mp.solutions.drawing_utils
#         self.tipIds=[4,8,12,16,20]
    
    def findHands(self,img,draw=True):
        imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        self.results=self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for hand_landmarks in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img,hand_landmarks,self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self,img,handno=0,draw=True):
        xList=[]
        yList=[]
        bbox=[]
        self.landmarks_list=[]
        if self.results.multi_hand_landmarks:
            myHand=self.results.multi_hand_landmarks[handno]
            for id,lm in enumerate(myHand.landmark):
                h,w,c=img.shape
                cx,cy=int(lm.x*w),int(lm.y*h)
                xList.append(cx)
                yList.append(cy)
                self.landmarks_list.append([id,cx,cy])
                if draw:
                    cv2.circle(img,(cx,cy),5,(255,0,255),cv2.FILLED)
            xmin,xmax=min(xList),max(xList)
            ymin,ymax=min(yList),max(yList)
            bbox=xmin,ymin,xmax,ymax
            
            if draw:
                cv2.rectangle(img,(bbox[0]-20,bbox[1]-20),(bbox[2]+20,bbox[3]+20),(0,255,0),2)
        return self.landmarks_list,bbox     
    def fingersUp(self):
        fingers=[]
        if self.landmarks_list[8][2] < self.landmarks_list[6][2]:
                fingers.append(1)
        else:
                fingers.append(0)
        if self.landmarks_list[12][2] < self.landmarks_list[10][2]:
                fingers.append(1)
        else:
                fingers.append(0)        
        return fingers

In [3]:
cap=cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
xp,yp=-1,-1
imgCanvas = np.zeros((720, 1280, 3), np.uint8)
color_selected=(255,0,255)
header1=cv2.imread('C:/Users/Shakshi/Downloads/p1.jpg')
header2=cv2.imread('C:/Users/Shakshi/Downloads/p2.jpg')
header3=cv2.imread('C:/Users/Shakshi/Downloads/p3.jpg')
header4=cv2.imread('C:/Users/Shakshi/Downloads/p4.jpg')
header=header2
detector=handDetector()
while True:
    success,img=cap.read()
    if not success:
        continue
    img=cv2.flip(img,1)
    img=detector.findHands(img,draw=True)
    landmarks_list,bbox=detector.findPosition(img,draw=False)
    if len(landmarks_list)!=0:
        fingers=detector.fingersUp()
        x1, y1 = landmarks_list[8][1:]
        x2, y2 = landmarks_list[12][1:]
        if fingers[0] and fingers[1]:
            xp,yp=-1,-1
            if y1 < 125:
                if  250<x1 <450 :#if i m clicking at purple brush
                    header =header2
                    color_selected = (255, 0, 255)
                elif 550 < x1 < 750:#if i m clicking at blue brush
                    header = header3
                    color_selected = (255, 105,65)
                elif 800 < x1 < 950:#if i m clicking at green brush
                    header = header4
                    color_selected = (0, 255, 0)
                elif 1050 < x1 < 1200:#if i m clicking at eraser
                    header = header1
                    color_selected = (0,0,0)
            cv2.rectangle(img, (x1, y1 - 25), (x2, y2 + 25), color_selected, cv2.FILLED)   
        if fingers[0] and fingers[1]==False:
            if(xp==-1 and yp ==-1):
                xp,yp=x1,y1
            cv2.circle(img, (x1, y1), 15, color_selected, cv2.FILLED)
            if color_selected==(0,0,0):
                cv2.line(img, (xp, yp),(x1,y1), color_selected,100)
                cv2.line(imgCanvas, (xp, yp), (x1, y1), color_selected, 100)
                xp,yp=x1,y1
            else:
                cv2.line(img, (xp, yp), (x1, y1), color_selected, 10)
                cv2.line(imgCanvas, (xp, yp), (x1, y1), color_selected, 10)
                xp,yp=x1,y1
    imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)
    _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)#on canvas all the region in which we drew is black and where it is black it is cosidered as white,it will create a mask
        
    imgInv = cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)#converting again to gray bcoz we have to add in a RGB image i.e img
    img = cv2.bitwise_and(img,imgInv)
        
        #add img and imgcanvas,by doing this we get colors on img
    img = cv2.bitwise_or(img,imgCanvas)


        #setting the header image
    img[0:125,0:1280]=header# on our frame we are setting our JPG image acc to H,W of jpg images

    cv2.imshow("Image", img)   
#     cv2.imshow('image1',imgCanvas)
#     cv2.imshow("image2",imgInv)
    if cv2.waitKey(1)==27:
        break  
cap.release()
cv2.destroyAllWindows()